In [13]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [14]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    #create a non-destructive copy
    movie = dict(movie)
    
    alt_titles ={}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
               'Hangul','Hebrew','Hepburn','Japanese','Literally',
               'Mandarin','McCune-Reischauer','Original title','Polish',
               'Revised Romanization','Romanized','Russian',
               'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    #merge column names
    def change_column_name(old_name, new_name):
       if old_name in movie:
           movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')


    return movie
    


In [41]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)
   
    # Open and read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)
        wiki_movies_df = pd.DataFrame(wiki_movies_raw)
        
    #write a list comprehension to filter out TV shows.
    wiki_tv = [tvshows for tvshows in wiki_movies_raw
        if 'Television series' in tvshows]
    
    # Write a list comprehension to filter out TV shows.                                    
#    wiki_movies = [movie for movie in wiki_movies_raw 
#        if ('Director' in movie or 'Directed by' in movie) 
#            and 'imdb_link' in movie 
#            and 'No. of episodes' not in movie]
    
    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies_raw]
                
    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
                 
    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except: 
        print("No link to extract")                

    # Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    
    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    ## Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
        
    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1,3}(?:,\d{3})+'
#    (?!\s[mb]illion)'

    # Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan


           
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars) 
    wiki_movies_df.drop('Box office', axis=1, inplace=True)    
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    
    # Convert an lists to strings
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
                      
    # Remove any values between a dollar sign and a hyphen for budgets given in ranges: 
    budget = budget.str.replace(r'\$.*[---](?![a-z])', '$', regex=True)
                      
    # Use the same patten matches, created to parse the box office data, and apply them
    # without modifications to the budget data
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE) 
    budget[~matches_form_one & ~matches_form_two]
                      
    # Remove the citation references with the following:
    budget = budget.str.replace(r'\[d+\]\s*', '')
    budget[~matches_form_one & ~matches_form_two]
                      
    #Ready to parse the budget values
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

       
    # Clean the release date column in the wiki_movies_df DataFrame.
    # Parsing the release date. First, make a variable that holds the non-null values of 
    # Release date in the DataFrame, converting lists to strings:
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Skill Drill - Parse the forms
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}' 
    
    # Extract the dates 
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
                      
    # Use the built-in to_datetime() function method in Pandas to parse the dates.
    # Different date formats, set the infer_datetime_format option to True.
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)                  
                      
    # Clean the running time column in the wiki_movies_df DataFrame.
    # Parse running time by first making a variable that holds the non-null values of Release date
    # in the DataFrame, converting lists to strings:
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Extract Values
    # Match all of the hour + minute patterns with one regular expression pattern.
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
                      
    # Create New DataFram of all strings, then convert them to numeric values.
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
                      
    # Now apply a function that will convert the hour capture groups and minute capture groups 
    # to minutes if the pure minutes capture group is zero, and save the output to wiki_movies_df:
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)                  
                      
    # Finally, drop Running time from the dataset.
    wiki_movies_df.drop('Running time', axis=1, inplace=True)  
 


  
    # 2. Clean the Kaggle metadata.
    # Look at the Kaggle.csv file data types
    kaggle_metadata.dtypes
    # check that all the values are either True or False.
    kaggle_metadata['adult'].value_counts()
    # Remove bad data
    kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]
    #Keep rows where the adult column is False, and then drop the adult column.

    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    # look at the values of the video column.
    kaggle_metadata['video'].value_counts()
    # Convert data types and create Boolean column
    
    
    kaggle_metadata['video'] == 'True'
    # Assign Boolean to video
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    # For the numeric columns, just use the to_numeric() method from Pandas.
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] 
    #= pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    # Finally, we need to convert release_date to datetime. Pandas has a built-in function: to_datetime().
#    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    
    # 3. Merged the two DataFrames into the movies DataFrame.
    # Print out a list of columns so we man identify redundancies.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
    #print(movies_df)
    movies_df[['title_wiki', 'title_kaggle']]
    
    movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]
    # Show any rows where title_kaggle is empty, drop wikipedia
    movies_df[(movies_df['title_kaggle'] == '') | (movies_df['title_kaggle'].isnull())]
    
    # We should investigate that wild outlier around 2006.
    movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')]
    # We'll have to drop that row from our DataFrame
    movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index
    # Then we can drop that row like this:
    movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)
    # Now, see if there are any null values:
    movies_df[movies_df['release_date_wiki'].isnull()]
    
    # For the language data, we'll compare the value counts of each.
    movies_df['Language'].value_counts()
    # We need to convert the lists in Language to tuples so that the value_counts() method will work.
    movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)

    
    
    # To see all the columns
#    pd.set_option("display.max_columns", None)
#    display(movies_df)

    # 4. Drop unnecessary columns from the merged DataFrame.
    # Put it all together. 
    # First, we'll drop the title_wiki, release_date_wiki, Language, and Production company(s) columns.
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language'], inplace=True)


    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
        df.drop(columns=wiki_column, inplace=True)
  
    
    
    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    # Don't forget, we need to convert lists to tuples for value_counts() to work.
     # Next, we'll make a function that fills in missing data for a column pair and then drops 
    # the redundant column.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    movies_df 
    
    
    
 
    # 7. Filter the movies DataFrame for specific columns.
    # Reorder columns in groups using LOC method.
    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
        if num_values == 1:
            print(col)

    
    movies_df = movies_df.loc[:,['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on']]
    
    # 8. Rename the columns in the movies DataFrame.
    # Finally, we need to rename the columns to be consistent.
    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)
    
    

    # 9. Transform and merge the ratings DataFrame.
    # Include the raw ratings data.
#    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()
    
    # Then we'll rename the "userId" column to "count.", NOTE: this is arbitrary.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                     .rename({'userId':'count'}, axis=1)

    # We can pivot this data so that movieId is the index, the columns will be all the rating values, and the rows will be the counts for each rating value.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                    .rename({'userId':'count'}, axis=1) \
                    .pivot(index='movieId',columns='rating', values='count')
    
    # We want to rename the columns so they're easier to understand. 
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns] 
    
    # This time, we need to use a left merge, since we want to keep everything in movies_df.
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

    # Finally, fill-in missing values instead of zeros, because not every movie got a rating for each rating level. 
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0) 
   
    movies_with_ratings_df
    return wiki_movies_df, movies_with_ratings_df, movies_df




In [42]:
  # 17. Create the path to your file directory and variables for the three files.\n",
#file_dir = '../Movies-ETL/Resources'
# The Wikipedia data\n",
#wiki_file = 'wikipedia-movies.json'
# The Kaggle metadata\n",
#kaggle_file = 'movies_metadata.csv'
# The MovieLens rating data.\n",
#ratings_file = 'ratings.csv'

In [43]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '/Users/m_201/Downloads/CLASSWORK_PROJECTS_2/Movies-ETL'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [44]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()


In [45]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [46]:
wiki_file.head

<bound method NDFrame.head of                                                     url    year  \
0     https://en.wikipedia.org/wiki/The_Adventures_o...  1990.0   
1     https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990.0   
2      https://en.wikipedia.org/wiki/Air_America_(film)  1990.0   
3       https://en.wikipedia.org/wiki/Alice_(1990_film)  1990.0   
4         https://en.wikipedia.org/wiki/Almost_an_Angel  1990.0   
...                                                 ...     ...   
7306    https://en.wikipedia.org/wiki/Holmes_%26_Watson  2018.0   
7307     https://en.wikipedia.org/wiki/Vice_(2018_film)  2018.0   
7308  https://en.wikipedia.org/wiki/On_the_Basis_of_Sex  2018.0   
7309  https://en.wikipedia.org/wiki/Destroyer_(2018_...  2018.0   
7310  https://en.wikipedia.org/wiki/Black_Mirror:_Ba...  2018.0   

                                  imdb_link                            title  \
0     https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
1    

In [47]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [48]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0


In [49]:
# 15. Check the movies_df DataFrame. 
print(movies_df)

Empty DataFrame
Columns: [imdb_id, kaggle_id, title, original_title, tagline, belongs_to_collection, wikipedia_url, imdb_link, runtime, budget, revenue, release_date, popularity, vote_average, vote_count, genres, original_language, overview, spoken_languages, country, production_companies, production_countries, distributor, producers, director, starring, cinematography, editors, writers, composers, based_on]
Index: []

[0 rows x 31 columns]


In [50]:
kaggle_file.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
